<a href="https://colab.research.google.com/github/karandomguy/iriscnn-dailymailsummariser/blob/tests/cnn_daily/mailtextsummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd

In [2]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail


newspaper-text-summarization-cnn-dailymail.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
! unzip newspaper-text-summarization-cnn-dailymail.zip

Archive:  newspaper-text-summarization-cnn-dailymail.zip
replace cnn_dailymail/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cnn_dailymail/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace cnn_dailymail/validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [8]:
train_df = pd.read_csv('cnn_dailymail/train.csv')
train_df

id  \
0       0001d1afc246a7964130f43ae940af6bc6c57f01   
1       0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2       00027e965c8264c35cc1bc55556db388da82b07f   
3       0002c17436637c4fe1837c935c04de47adb18e9a   
4       0003ad6ef0c37534f80b55b4235108024b407f0b   
...                                          ...   
287108  fffdfb56fdf1a12d364562cc2b9b1d4de7481dee   
287109  fffeecb8690b85de8c3faed80adbc7a978f9ae2a   
287110  ffff5231e4c71544bc6c97015cdb16c60e42b3f4   
287111  ffff924b14a8d82058b6c1c5368ff1113c1632af   
287112  ffffd563a96104f5cf4493cfa701a65f31b06abf   

                                                  article  \
0       By . Associated Press . PUBLISHED: . 14:11 EST...   
1       (CNN) -- Ralph Mata was an internal affairs li...   
2       A drunk driver who killed a young woman in a h...   
3       (CNN) -- With a breezy sweep of his pen Presid...   
4       Fleetwood are the only team still to have a 10...   
...                                                   ...   
287108  By . James Rush . Former first daughter Chelse...   
287109  An apologetic Vanilla Ice has given his first ...   
287110  America's most lethal sniper claimed he wished...   
287111  By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...   
287112  (CNN)Former Florida Gov. Jeb Bush has decided ...   

                                               highlights  
0       Bishop John Folda, of North Dakota, is taking ...  
1       Criminal complaint: Cop used his role to help ...  
2       Craig Eccleston-Todd, 27, had drunk at least t...  
3       Nina dos Santos says Europe must be ready to a...  
4       Fleetwood top of League One after 2-0 win at S...  
...                                                   ...  
287108  Chelsea Clinton said question of running for o...  
287109  Vanilla Ice, 47 - real name Robert Van Winkle ...  
287110  America's most lethal sniper made comment in i...  
287111  A swarm of more than one million has crossed b...  
287112  Other 2016 hopefuls maintain that Bush's annou...  

[287113 rows x 3 columns]

In [9]:
test_data = pd.read_csv('cnn_dailymail/test.csv')

In [10]:
train_df = train_df.drop(['id'],axis=1)

In [11]:
train_data=train_df

In [12]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.drop(['id'], axis=1)
test_data = test_data.reset_index(drop=True)

# **Abstractive Summarization using Pegasus(Hugging Face transformer)**



In [14]:
from summarizer import Summarizer
from pprint import pprint

In [15]:
!pip install transformers
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
train_check=train_data[1:5]

In [26]:
train_check

article  \
1  (CNN) -- Ralph Mata was an internal affairs li...   
2  A drunk driver who killed a young woman in a h...   
3  (CNN) -- With a breezy sweep of his pen Presid...   
4  Fleetwood are the only team still to have a 10...   

                                          highlights  
1  Criminal complaint: Cop used his role to help ...  
2  Craig Eccleston-Todd, 27, had drunk at least t...  
3  Nina dos Santos says Europe must be ready to a...  
4  Fleetwood top of League One after 2-0 win at S...

In [17]:
from transformers import pipelines

In [18]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [22]:
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [23]:
model= PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [45]:
tokens = tokenizer(train_check['article'][2], truncation=True, padding="longest", return_tensors="pt")

In [46]:
tokens

{'input_ids': tensor([[  114, 11754,  1712,   170,  3040,   114,   758,  1590,   115,   114,
           693,   121,   661,  4897,   277,  3782,   169,   944,   685,   148,
           174, 33318,   118,  1029,   231,   107,   110, 68797,   110, 13521,
         36521,  2035,   121,   144, 33967,   108,  8358,   140,  1528,   238,
           135,   114,   565,   134,   114,  7821,   173,   178,   915,   114,
          1352,  1285,   107,   130,   178,   140,   961,   132, 40698,   112,
           126,   108,   178, 40182,   316,   482,   109,  1018,   277,  1528,
          1344,   114, 11622,   111, 24354,   190,   110, 93297,  1560,   415,
           123,   116,   439,   792,   109,   176,   230,   107,   110, 68797,
           110, 13521, 36521,  2035,   121,   144, 33967,   108,  2684,   143,
         12232,   158,   140,   303,   169,   944,   685,   173,   178, 14726,
           693,   121,   661,   190,   109,   439,   270,  3830,   141,   110,
         93297,  1560,   415,   108,  

In [47]:
summary = model.generate(**tokens)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [48]:
tokenizer.decode(summary[0])

'<pad>A drunk driver who killed a young woman in a head-on crash while checking his mobile phone has been jailed for six years.</s>'

In [51]:
train_check['hightlights'][2]

"craig eccleston-todd, 27, had drunk at least three pints before driving car .\nwas using phone when he veered across road in yarmouth, isle of wight .\ncrashed head-on into 28-year-old rachel titley's car, who died in hospital .\npolice say he would have been over legal drink-drive limit at time of crash .\nhe was found guilty at portsmouth crown court of causing death by dangerous driving ."

In [40]:

!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(tokenizer.decode(summary[0]),
                      train_check['highlights'][2])

In [43]:
scores

{'rouge1': Score(precision=0.11428571428571428, recall=0.17391304347826086, fmeasure=0.13793103448275862),
 'rougeL': Score(precision=0.05714285714285714, recall=0.08695652173913043, fmeasure=0.06896551724137931)}

# **Abstractive Summarisation using BeautifulSoup**

In [52]:
from bs4 import BeautifulSoup
import requests

In [53]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [76]:
ARTICLE=train_check['article'][2]
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [87]:
max_chunk = 500
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [88]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [89]:
output=str(res[0])

In [90]:
train_check['highlights'][2]

"Craig Eccleston-Todd, 27, had drunk at least three pints before driving car .\nWas using phone when he veered across road in Yarmouth, Isle of Wight .\nCrashed head-on into 28-year-old Rachel Titley's car, who died in hospital .\nPolice say he would have been over legal drink-drive limit at time of crash .\nHe was found guilty at Portsmouth Crown Court of causing death by dangerous driving ."

In [91]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(output,
                      train_check['highlights'][2])

In [92]:
scores

{'rouge1': Score(precision=0.5142857142857142, recall=0.5, fmeasure=0.5070422535211268),
 'rougeL': Score(precision=0.2857142857142857, recall=0.2777777777777778, fmeasure=0.28169014084507044)}